In [1]:
import pandas as pd
import re

In [13]:
# Path to Skyline output CSV
skyline_csv = "C:/Users/49152/Desktop/Repositories/Multiomics_study/Fluxomics/Skyline_trial/Example_output_from_Skyline/Transition_Results.csv"

#Path to output directory
out_dir = "C:/Users/49152/Downloads/Fluxomics/PICor/Input/"

# Read Skyline CSV
df = pd.read_csv(skyline_csv)

df

,Molecule,Molecule List Name,Replicate Name,Precursor Mz,Precursor Adduct,Precursor Charge,Fragment Ion,Product Mz,Product Adduct,Product Charge,Retention Time,Area,Background,Peak Rank
0,Pyruvate,molecules1,250509_MHO06AI_CoA_1,87.008768,[M-H],-1,precursor,87.008768,[M-H],-1,0.47,11440586,1021783,1
1,Pyruvate,molecules1,250509_MHO06AI_CoA_2,87.008768,[M-H],-1,precursor,87.008768,[M-H],-1,0.47,64616940,5157005,1
2,Pyruvate,molecules1,250509_MHO06AI_CoA_3,87.008768,[M-H],-1,precursor,87.008768,[M-H],-1,0.47,8951289,1136696,1
3,Pyruvate,molecules1,250509_MHO06AI_CoA_4,87.008768,[M-H],-1,precursor,87.008768,[M-H],-1,0.47,18313592,1614532,1
4,Pyruvate,molecules1,250509_MHO06AI_CoA_5,87.008768,[M-H],-1,precursor,87.008768,[M-H],-1,0.47,1743339,4802845,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689,AcetylCoA,molecules1,250509_MHO06AI_CoA_116,817.148692,[M9C13-H],-1,precursor,817.148692,[M9C13-H],-1,2.18,472,0,1
1690,AcetylCoA,molecules1,250509_MHO06AI_CoA_117,817.148692,[M9C13-H],-1,precursor,817.148692,[M9C13-H],-1,2.19,655,0,1
1691,AcetylCoA,molecules1,250509_MHO06AI_CoA_118,817.148692,[M9C13-H],-1,precursor,817.148692,[M9C13-H],-1,2.21,658,0,1
1692,AcetylCoA,molecules1,250509_MHO06AI_CoA_119,817.148692,[M9C13-H],-1,precursor,817.148692,[M9C13-H],-1,2.16,110,0,1


In [15]:
def get_isotope(adduct):
    """
    Convert Precursor Adduct string to isotope number.
    Examples:
    [M-H] -> 0
    [MC13-H] -> 1
    [M2C13-H] -> 2
    """
    adduct = str(adduct)
    if adduct == "[M-H]":
        return 0
    match = re.match(r"\[M(\d*)C13-H\]", adduct)
    if match:
        num = match.group(1)
        return int(num) if num else 1
    return None

In [17]:
df["isotope"] = df["Precursor Adduct"].apply(get_isotope)
df["isotope"]

0       0
1       0
2       0
3       0
4       0
       ..
1689    9
1690    9
1691    9
1692    9
1693    9
Name: isotope, Length: 1694, dtype: int64

In [19]:
# Process each metabolite separately
metabolites = df["Molecule"].unique()

In [21]:
for mol in metabolites:
    df_mol = df[df["Molecule"] == mol].copy()
    
    # Determine maximum isotope number for this metabolite
    max_iso = df_mol["isotope"].max()
    
    # Map isotope number to column names dynamically
    iso_columns = {}
    for i in range(1, max_iso+1):
        iso_columns[i] = f"{i}C13"
    iso_columns[0] = "No Label"
    
    # Pivot
    pivoted = df_mol.pivot_table(
        index=["Replicate Name", "Molecule"],
        columns="isotope",
        values="Area",
        aggfunc="first"
    ).rename(columns=iso_columns)
    
    # Fill missing isotopes with NA
    for col in iso_columns.values():
        if col not in pivoted.columns:
            pivoted[col] = "NA"
    
    # Reset index
    pivoted = pivoted.reset_index()
    
    # Add index column
    pivoted.insert(0, "index", range(1, len(pivoted)+1))
    
    # Rename columns to match PICor
    pivoted.rename(columns={"Replicate Name": "filename", "Molecule": "compound"}, inplace=True)
    
    # Reorder columns
    ordered_cols = ["index", "filename", "compound"] + [iso_columns[i] for i in sorted(iso_columns.keys())]
    pivoted = pivoted[ordered_cols]
    
    # Save CSV
    out_csv = out_dir + f"{mol}_picor.csv"
    pivoted.to_csv(out_csv, index=False)
    print(f"Saved {out_csv}")

Saved C:/Users/49152/Downloads/Fluxomics/PICor/Input/Pyruvate_picor.csv
Saved C:/Users/49152/Downloads/Fluxomics/PICor/Input/AcetylCoA_picor.csv
